In [6]:
import json
import logging
import random
from copy import deepcopy
from pathlib import Path
import re

import torch
from joblib import Parallel, delayed
from poke_env.environment.battle import Battle
from tqdm.auto import tqdm

from alphagogoat.catalogs import MoveEnum
from alphagogoat.constants import LSTM_INPUT_SIZE, DEVICE
from alphagogoat.delphox import Delphox, train, evaluate
from alphagogoat.utils import move_to_pred_vec_index

LOGGER = logging.getLogger('poke-env')

def process_input_log(log):
    """
    >>> log = Path("cache/replays/gen8randombattle-1872565566.log").read_text()
    >>> process_input_log(log)
    """
    input_log = log['inputlog']
    input_log = input_log.split('\n')
    start = 0
    for line in input_log:
        if line.startswith('>p1'):
            break
        start += 1

    input_log = input_log[start:]
    out1 = []
    out2 = []

    for i in range(len(input_log) - 1):
        curr_line = input_log[i]
        next_line = input_log[i+1]
        if curr_line.startswith('>p1') and next_line.startswith('>p2'): # that means this is a normal turn with no fainting or anything
            out_me = torch.zeros(len(MoveEnum) + 1)
            out_them = torch.zeros(len(MoveEnum) + 1)
            curr_line = curr_line.split(' ')
            next_line = next_line.split(' ')

            if curr_line[1] == 'move':
                out_me[move_to_pred_vec_index(curr_line[2])] = 1
            elif curr_line[1] == 'switch':
                print(curr_line)
                out_me[-1] = 1

            if next_line[1] == 'move':
                out_them[move_to_pred_vec_index(next_line[2])] = 1
            elif next_line[1] == 'switch':
                print(next_line)
                out_them[-1] = 1
            i += 1
            out1.append(out_me)
            out2.append(out_them)
        else:
            continue

    return out1, out2

def make_data(filepath):
    with open(filepath) as f:
        replay = json.load(f)
    history = replay['log'].split('\n')
    battles = []
    b = Battle(replay['id'], replay['p1'], LOGGER, 8)
    b._opponent_username = replay['p2']
    mon1, mon2 = None, None
    for line in history:
        try:
            b._parse_message(line.split('|'))
            # print(b.turn)
            # print(b.active_pokemon)
            # print(b.opponent_active_pokemon)
            if mon1 is None:
                mon1 = b.active_pokemon
            if mon2 is None:
                mon2 = b.opponent_active_pokemon
            if line.split('|')[1] == 'turn':
                # monkey patch issue where fainted pokemon are immediately replaced the same turn
                b.active_mon = mon1
                b.opponent_active_mon = mon2
                battles.append(deepcopy(b))
                mon1, mon2 = None, None
        except:
            continue
    move1, move2 = process_input_log(replay)
    return battles, move1, move2

def move_to_pred_vec_index(m):
    return MoveEnum[re.sub(r"\s|-|'", "", m.lower())].value - 1


def pred_vec_to_string(pred: torch.Tensor):
    i = pred.argmax().item()
    if i == len(MoveEnum):
        return 'switch'
    else:
        return MoveEnum(i + 1).name

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'victini'

In [80]:
def process_line(line: str):
    if "switch" in line:
        pokemon_switch = re.search(r"p[12]a: (.*?)\|", line).groups(0)[0]
        pokemon_switch = re.sub(r"[-’\s\.:]", "", pokemon_switch.lower())
        return ("switch", pokemon_switch)
    else:
        pokemon_move = re.search(r"\|([A-Z].*?)\|", line).groups(0)[0]
        pokemon_move = re.sub(r"\s|-|'", "", pokemon_move.lower())
        return ("move", pokemon_move)

def get_histories(filepath: str):
    history1, history2 = [], []
    with open(filepath) as f:
        data = json.load(f)
    turn_texts = data['log'].split('|turn|')[1:]
    for text in turn_texts:
        matches = re.findall(r"(\|[ms].+\|)", text)
        for m in matches:
            if "|move|p1a:" in m or "|switch|p1a:" in m:
                cooked = process_line(m)
                history1.append(cooked)
                break
        for m in matches:
            if "|move|p2a:" in m or "|switch|p2a:" in m:
                cooked = process_line(m)
                history2.append(cooked)
                break
    return history1, history2

In [81]:
history1, history2 = get_histories("cache/replays/gen8randombattle-1872565566.json")
history1

[('switch', 'suicune'),
 ('switch', 'buzzwole'),
 ('move', 'stoneedge'),
 ('move', 'stoneedge'),
 ('move', 'stoneedge'),
 ('move', 'stoneedge'),
 ('switch', 'torkoal'),
 ('move', 'stealthrock'),
 ('move', 'lavaplume'),
 ('move', 'solarbeam'),
 ('switch', 'cryogonal'),
 ('move', 'rapidspin'),
 ('move', 'earthquake'),
 ('move', 'maxknuckle'),
 ('move', 'maxquake'),
 ('switch', 'gyarados'),
 ('switch', 'krookodile'),
 ('move', 'earthquake'),
 ('move', 'calmmind'),
 ('move', 'rest'),
 ('move', 'sleeptalk'),
 ('move', 'sleeptalk'),
 ('move', 'scald')]

In [82]:
turns, moves1, moves2 = make_data("cache/replays/gen8randombattle-1872565566.json")

['>p1', 'switch', '4']
['>p1', 'switch', '5']
['>p1', 'switch', '3']
['>p1', 'switch', '3']


In [83]:
for turn, m1, m2 in zip(turns, history1, history2):
    print(f"Turn {turn.turn}:"
          f"\n\t {turn.active_pokemon}:\t{m1}"
          f"\n\t {turn.opponent_active_pokemon}:\t{m2}")

Turn 1:
	 cryogonal (pokemon object) [Active: True, Status: None]:	('switch', 'suicune')
	 ninetales (pokemon object) [Active: True, Status: None]:	('move', 'nastyplot')
Turn 2:
	 suicune (pokemon object) [Active: True, Status: None]:	('switch', 'buzzwole')
	 ninetales (pokemon object) [Active: True, Status: None]:	('move', 'solarbeam')
Turn 3:
	 buzzwole (pokemon object) [Active: True, Status: None]:	('move', 'stoneedge')
	 ninetales (pokemon object) [Active: True, Status: None]:	('move', 'scorchingsands')
Turn 4:
	 buzzwole (pokemon object) [Active: True, Status: BRN]:	('move', 'stoneedge')
	 ninetales (pokemon object) [Active: True, Status: None]:	('move', 'fireblast')
Turn 5:
	 buzzwole (pokemon object) [Active: True, Status: BRN]:	('move', 'stoneedge')
	 ninetales (pokemon object) [Active: True, Status: None]:	('move', 'fireblast')
Turn 6:
	 buzzwole (pokemon object) [Active: True, Status: BRN]:	('move', 'stoneedge')
	 ninetales (pokemon object) [Active: True, Status: None]:	('swi

In [1]:
from poke_env.environment.pokemon import Pokemon
from alphagogoat.constants import *
import alphagogoat.embedder as embedder
emb = embedder.Embedder()
import alphagogoat.victini as vc
from alphagogoat.utils import make_delphox_data
from pathlib import Path
json_files = [filepath for filepath in Path("cache/replays").iterdir() if filepath.name.endswith('.json')]
test = make_delphox_data(json_files[0])
battle = test[0][6]

0 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) sylveon (pokemon object) [Active: True, Status: None] ['calmmind', 'hypervoice', 'mysticalfire', 'protect', 'psyshock', 'shadowball', 'wish']
0 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) shedinja (pokemon object) [Active: True, Status: None] ['poltergeist', 'shadowsneak', 'swordsdance', 'willowisp', 'xscissor']
4 tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]) shedinja (pokemon object) [Active: True, Status: None] ['poltergeist', 'shadowsneak', 'swordsdance', 'willowisp', 'xscissor']
1 tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) sylveon (pokemon object) [Active: True, Status: None] ['calmmind', 'hypervoice', 'mysticalfire', 'protect', 'psyshock', 'shadowball', 'wish']
2 tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) sylveon (pokemon object) [Active: True, Status: None] ['calmmind', 'hypervoice', 'mysticalfire', 'protect', 'psyshock', 'shadowball', 'wish']
1 tensor([0., 1., 0., 0., 0

In [4]:
battle.active_pokemon.moves

{}

In [6]:
vc.computeFuture(battle, (('bellossom', 'quiverdance'), ('sylveon', 'calmmind')))

tensor([[ 1.0000e+00,  1.6000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.0000e+00,  3.3000e+01,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  1.7800e+02,  2.0300e+02,
          2.5700e+02,  1.9500e+02,  2.1100e+02,  1.2900e+02,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
          2.5700e+02,  1.0000e+00,  0.0000e+00,  8.2000e+01,  0.0000e+00,
          0.0000e+00,  1.0000e+01, -1.0000e+00, -1.0000e+00,  1.0000e+00,
          1.5400e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          4.3700e-01,  3.3000e+01,  3.7580e-01,  3.4000e+01,  1.8720e-01,
          1.0000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  1.9000e+02,  1.8300e+02,  2.7800e+02,
          2.1300e+02,  1.8700e+02,  1.

In [15]:
from poke_env.environment.status import Status

In [16]:
Status['FNT']

<Status.FNT: 2>

In [7]:
def pass_around(b):
    b.team['gigalith'] = Pokemon(species='gigalith', gen=8)

In [8]:
d = pass_around(battle)

In [10]:
battle.team
d.team

AttributeError: 'NoneType' object has no attribute 'team'